# 라이브러리 Install

In [1]:
%pip install --upgrade pip

%pip install qdrant_client rank_bm25 sentence_transformers langchain_text_splitters pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 라이브러리 Import

In [2]:
from collections import defaultdict
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, SparseVectorParams, PointStruct, SparseVector
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List

import pandas as pd
import numpy as np

print("✅ 라이브러리 Import 완료")

✅ 라이브러리 Import 완료


# 임베딩 모델, 벡터 DB 초기화

  📊 사용 가능한 로컬 임베딩 모델

  | 모델                                    | 차원  | 속도     | 품질      | 용도                    |
  |---------------------------------------|-----|--------|---------|-----------------------|
  | all-MiniLM-L6-v2                      | 384 | ⚡⚡⚡ 빠름 | ⭐⭐⭐ 좋음  | 빠른 테스트                |
  | all-mpnet-base-v2                     | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 추천 (Vertex AI와 동일 차원) |
  | paraphrase-multilingual-mpnet-base-v2 | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 한국어 지원 좋음             |

In [3]:
# ========================================
# 1. 초기화
# ========================================
model = "all-MiniLM-L6-v2"
sentenceTransformer = SentenceTransformer(model)
print(f"✅ SentenceTransformer 모델 초기화 완료 ({model})")

qdrantClient = QdrantClient(":memory:")
print("✅ Qdrant 생성 완료 (메모리 모드)")

✅ SentenceTransformer 모델 초기화 완료 (all-MiniLM-L6-v2)
✅ Qdrant 생성 완료 (메모리 모드)


# 컬렉션 생성

In [4]:
# ========================================
# 2. 컬렉션 생성
# ========================================
collection_name="test"

if qdrantClient.collection_exists(collection_name=collection_name):
    qdrantClient.delete_collection(collection_name=collection_name)
    print("✅ Qdrant 기존 컬렉션 삭제 완료")

qdrantClient.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=384, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams()
    }
)
print("✅ Qdrant 컬렉션 생성 완료")

✅ Qdrant 컬렉션 생성 완료


# 테스트 데이터 생성 (청킹 기반 워크플로우)

**워크플로우**: 긴 문서 → 청킹 → 임베딩 → 벡터 DB 저장

In [5]:
# ========================================
# 3-1. 긴 문서 샘플 정의
# ========================================

# 음식점 소개 긴 문서들 (여러 매장)
raw_documents = [
    """
    미쉐린 3스타에 빛나는 '라떼르'는 서울 강남구 청담동에 위치한 프렌치 파인다이닝 레스토랑입니다. 
    셰프 박준우는 프랑스 미슐랭 3스타 레스토랑 '피에르 가니에르'에서 10년간 수련한 후 한국으로 돌아와 
    한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처 메뉴는 '제주 흑돼지와 송로버섯을 곁들인 부르고뉴 스타일 스튜'입니다.
    """,
    """
    블루리본에 선정된 '일미정'은 전통 일식 오마카세 전문점입니다. 
    오마카세는 셰프에게 맡긴다는 의미로, 당일 공수한 제철 생선으로 구성됩니다.
    노량진 수산시장에서 새벽에 직접 공수한 신선한 해산물을 사용하며,
    참치, 광어, 성게 등 10가지 이상의 스시 코스를 제공합니다.
    """,
    """
    강남역 근처 '브런치클럽'은 주말마다 2시간 이상 웨이팅이 발생하는 인기 맛집입니다.
    SNS에서 화제가 된 계절 과일 팬케이크와 아보카도 토스트가 시그니처 메뉴입니다.
    인테리어는 북유럽 스타일로 밝고 감각적이며, 사진 촬영하기 좋은 곳으로 유명합니다.
    예약은 불가하며 현장 웨이팅만 가능합니다.
    """,
    """
    한남동의 '미트하우스'는 45일 숙성육 전문 스테이크하우스입니다.
    미국산 프라임 등급 채끝, 토마호크 등을 직접 숙성실에서 관리하며,
    1인 1스테이크 정책으로 최고의 품질을 유지합니다.
    와인 페어링도 인기이며, 전문 소믈리에가 고기에 맞는 와인을 추천해줍니다.
    """,
    """
    이태원의 '파스타팩토리'는 홈메이드 파스타 전문점으로 블루리본에 선정되었습니다.
    매일 아침 수제로 만드는 탈리아텔레, 라비올리 등의 생면을 사용하며,
    이탈리아 현지 레시피를 고수합니다. 트러플 크림 파스타와 해산물 리조또가 인기 메뉴입니다.
    점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다.
    """
]

print(f"✅ {len(raw_documents)}개 긴 문서 준비 완료")
for i, doc in enumerate(raw_documents, 1):
    print(f"   매장 {i}: {len(doc)} 자")

✅ 5개 긴 문서 준비 완료
   매장 1: 223 자
   매장 2: 174 자
   매장 3: 186 자
   매장 4: 167 자
   매장 5: 184 자


In [6]:
# ========================================
# 3-2. 청킹 기법 선택 및 docs 생성 (LangChain)
# ========================================

# LangChain의 RecursiveCharacterTextSplitter 사용
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=30,
    length_function=len,
    separators=[
        "\n\n",  # 문단
        "\n",    # 줄바꿈
        ". ",    # 문장 (마침표 + 공백)
        ".",     # 마침표
        " ",     # 공백
        "",      # 문자
    ]
)

print("✅ LangChain RecursiveCharacterTextSplitter 초기화")
print(f"   • chunk_size: 150")
print(f"   • chunk_overlap: 30")
print(f"   • separators: 문단 → 문장 → 공백 → 문자 순서\n")

# 모든 문서를 청킹하여 docs 배열 생성
docs = []
doc_metadata = []  # 어느 원본 문서에서 왔는지 추적

for doc_idx, raw_doc in enumerate(raw_documents, 1):
    # LangChain으로 청킹
    chunks = text_splitter.split_text(raw_doc)
    
    print(f"매장 {doc_idx}: {len(raw_doc)}자 → {len(chunks)}개 청크로 분할")
    
    for chunk_idx, chunk in enumerate(chunks, 1):
        docs.append(chunk)
        doc_metadata.append({
            "shop_id": doc_idx,
            "chunk_id": chunk_idx,
            "original_length": len(raw_doc)
        })
        print(f"  청크 {chunk_idx}: {len(chunk)}자 - {chunk[:50]}...")

print(f"\n{'='*60}")
print(f"✅ 총 {len(docs)}개 청크 생성 완료 (원본 {len(raw_documents)}개 문서)")
print(f"{'='*60}")

✅ LangChain RecursiveCharacterTextSplitter 초기화
   • chunk_size: 150
   • chunk_overlap: 30
   • separators: 문단 → 문장 → 공백 → 문자 순서

매장 1: 223자 → 2개 청크로 분할
  청크 1: 117자 - 미쉐린 3스타에 빛나는 '라떼르'는 서울 강남구 청담동에 위치한 프렌치 파인다이닝 레스토랑...
  청크 2: 90자 - 한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처...
매장 2: 174자 → 2개 청크로 분할
  청크 1: 124자 - 블루리본에 선정된 '일미정'은 전통 일식 오마카세 전문점입니다. 
    오마카세는 셰프에...
  청크 2: 35자 - 참치, 광어, 성게 등 10가지 이상의 스시 코스를 제공합니다....
매장 3: 186자 → 2개 청크로 분할
  청크 1: 97자 - 강남역 근처 '브런치클럽'은 주말마다 2시간 이상 웨이팅이 발생하는 인기 맛집입니다.
  ...
  청크 2: 74자 - 인테리어는 북유럽 스타일로 밝고 감각적이며, 사진 촬영하기 좋은 곳으로 유명합니다.
   ...
매장 4: 167자 → 2개 청크로 분할
  청크 1: 111자 - 한남동의 '미트하우스'는 45일 숙성육 전문 스테이크하우스입니다.
    미국산 프라임 등...
  청크 2: 41자 - 와인 페어링도 인기이며, 전문 소믈리에가 고기에 맞는 와인을 추천해줍니다....
매장 5: 184자 → 2개 청크로 분할
  청크 1: 142자 - 이태원의 '파스타팩토리'는 홈메이드 파스타 전문점으로 블루리본에 선정되었습니다.
    매...
  청크 2: 27자 - 점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다....

✅ 총 10개 청크 생성 완료 (원본 5개 문서)


# 💡 청킹 기반 워크플로우 요약

**전체 흐름**:
```
1. 긴 원본 문서 (raw_documents)
   ↓
2. 청킹 처리 (LangChain RecursiveCharacterTextSplitter)
   - chunk_size=150, overlap=30
   - 구분자 우선순위: 문단 → 문장 → 공백 → 문자
   ↓
3. 청크 배열 생성 (docs + doc_metadata)
   ↓
4. Dense 임베딩 (SentenceTransformer)
   ↓
5. Sparse 인덱싱 (BM25)
   ↓
6. 벡터 DB 저장 (Qdrant)
   - Dense + Sparse 벡터
   - Metadata (shop_id, chunk_id 등)
   ↓
7. Hybrid Search (Dense + Sparse + RRF)
```

**LangChain 사용의 장점**:
- ✅ 검증된 청킹 알고리즘 (프로덕션 레벨)
- ✅ 자연스러운 문서 경계 유지 (문단/문장 우선 분할)
- ✅ 다양한 언어 지원 및 커스터마이징 가능
- ✅ 각 청크가 어떤 원본 문서에서 왔는지 추적 가능
- ✅ 검색 시 관련 청크만 반환하여 정확도 향상

# 청킹된 데이터 임베딩 & BM25 인덱싱

- 청킹된 docs를 Dense(임베딩) + Sparse(BM25)로 처리

In [ ]:
# Dense 임베딩
dense_embeddings = sentenceTransformer.encode(docs)
print(f"✅ Dense 임베딩 생성: {dense_embeddings.shape}")

# BM25 인덱스 (전체 문서로 학습)
bm25 = BM25Okapi([doc.split() for doc in docs])
print(f"✅ BM25 인덱싱 완료")

✅ Dense 임베딩 생성: (10, 384)
✅ BM25 인덱싱 완료


# 테스트 데이터 저장 (Dense, Sparse)

In [ ]:
# ========================================
# 5. 청킹된 데이터를 Qdrant에 저장
# ========================================

# Qdrant에 삽입 (metadata 포함)
points = []
for idx, (doc, dense, meta) in enumerate(zip(docs, dense_embeddings, doc_metadata)):
    sparse_scores = bm25.get_scores(doc.split())
    sparse_idx = np.where(sparse_scores > 0)[0]

    points.append(PointStruct(
        id=idx,
        vector={
            "dense": dense.tolist(),
            "sparse": SparseVector(
                indices=sparse_idx.tolist(),
                values=sparse_scores[sparse_idx].tolist(),
            )
        },
        payload={
            "text": doc,
            "shop_name": f"매장_{meta['shop_id']}",
            "shop_id": meta['shop_id'],
            "chunk_id": meta['chunk_id'],
            "original_length": meta['original_length']
        }
    ))

qdrantClient.upsert(collection_name=collection_name, points=points)
print(f"\n{'='*60}")
print(f"✅ {len(points)}개 청크 포인트 삽입 완료")
print(f"   • 원본 문서: {len(raw_documents)}개")
print(f"   • 총 청크: {len(docs)}개")
print(f"   • Dense 벡터 차원: {len(dense_embeddings[0])}")
print(f"{'='*60}")

# Hybrid Search (Dense + Sparse + RRF) 함수 정의

In [ ]:
# ========================================
# 5-1. Hybrid Search 함수 (RRF) - 업데이트
# ========================================
def retrieve_similar_examples_hybrid(query, collection_name, top_k=2):
    # 1. query -> Dense 임베딩 생성
    query_dense_embedding = sentenceTransformer.encode(query)

    # 2. query -> Dense 검색
    dense_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=query_dense_embedding.tolist(),
        using="dense",
        limit=10
    )

    # 3. query -> Sparse 벡터 생성 (BM25)
    query_tokens = query.split()
    query_sparse_scores = bm25.get_scores(query_tokens)
    query_sparse_idx = np.where(query_sparse_scores > 0)[0]

    # 4. query -> Sparse 검색
    sparse_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=query_sparse_idx.tolist(),
            values=query_sparse_scores[query_sparse_idx].tolist()
        ),
        using="sparse",
        limit=10
    )

    # 5. RRF 융합 (k=60은 표준 값)
    scores = defaultdict(float)
    k = 60

    # 6. Dense 점수 반영
    for rank, hit in enumerate(dense_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 7. Sparse 점수 반영
    for rank, hit in enumerate(sparse_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 8. 상위 top_k개 선택
    top_ids = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # 9. 실제 데이터 가져오기 (metadata 포함)
    final_results = []
    for point_id, rrf_score in top_ids:
        point = qdrantClient.retrieve(collection_name=collection_name, ids=[point_id])[0]
        final_results.append({
            "shop_name": point.payload.get("shop_name", "Unknown"),
            "text": point.payload.get("text", ""),
            "shop_id": point.payload.get("shop_id"),
            "chunk_id": point.payload.get("chunk_id"),
            "score": rrf_score,
            "id": point_id
        })

    return final_results

print("✅ Hybrid Search 함수 업데이트 완료 (metadata 포함)")

✅ Hybrid Search 함수 업데이트 완료 (metadata 포함)


# Hybrid Search 테스트 (청킹된 데이터로)

In [ ]:
# ========================================
# 6. 청킹된 데이터로 Hybrid Search 테스트
# ========================================

queries = [
   "프렌치 레스토랑",
   "스시 오마카세",
   "웨이팅 맛집"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"🔍 검색 쿼리: '{query}'")
    print(f"{'='*60}")

    results = retrieve_similar_examples_hybrid(query, collection_name, top_k=3)

    for idx, result in enumerate(results, 1):
        print(f"\n{idx}. {result['shop_name']} (Score: {result['score']:.4f})")
        print(f"   [청크 정보] shop_id={result.get('shop_id', 'N/A')}, chunk_id={result.get('chunk_id', 'N/A')}")
        print(f"   {result['text'][:100]}...")


🔍 검색 쿼리: '프렌치 레스토랑'

1. 매장_5 (Score: 0.0320)
   [청크 정보] shop_id=5, chunk_id=2
   점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다....

2. 매장_1 (Score: 0.0315)
   [청크 정보] shop_id=1, chunk_id=2
   한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처 메뉴는 '제주 흑돼지와 송로버섯을 곁들인 부르고뉴 스타일 스튜'입니다....

3. 매장_3 (Score: 0.0312)
   [청크 정보] shop_id=3, chunk_id=1
   강남역 근처 '브런치클럽'은 주말마다 2시간 이상 웨이팅이 발생하는 인기 맛집입니다.
    SNS에서 화제가 된 계절 과일 팬케이크와 아보카도 토스트가 시그니처 메뉴입니다....

🔍 검색 쿼리: '스시 오마카세'

1. 매장_2 (Score: 0.0318)
   [청크 정보] shop_id=2, chunk_id=2
   참치, 광어, 성게 등 10가지 이상의 스시 코스를 제공합니다....

2. 매장_2 (Score: 0.0313)
   [청크 정보] shop_id=2, chunk_id=1
   블루리본에 선정된 '일미정'은 전통 일식 오마카세 전문점입니다. 
    오마카세는 셰프에게 맡긴다는 의미로, 당일 공수한 제철 생선으로 구성됩니다.
    노량진 수산시장에서 새...

3. 매장_4 (Score: 0.0303)
   [청크 정보] shop_id=4, chunk_id=1
   한남동의 '미트하우스'는 45일 숙성육 전문 스테이크하우스입니다.
    미국산 프라임 등급 채끝, 토마호크 등을 직접 숙성실에서 관리하며,
    1인 1스테이크 정책으로 최고의...

🔍 검색 쿼리: '웨이팅 맛집'

1. 매장_5 (Score: 0.0164)
   [청크 정보] shop_id=5, chunk_id=2
   점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다....

2. 매장_3 

# Dense vs Sparse vs Hybrid Search (RFF) 비교

In [ ]:
# ========================================
# 6. Dense vs Sparse vs Hybrid 비교
# ========================================

print(f"\n\n{'='*60}")
print("📊 Dense vs Sparse vs Hybrid 비교")
print(f"{'='*60}")

test_queries = [
    "미쉐린",         # 정확한 키워드
    "고급 레스토랑",    # 의미적 검색
    "스시",           # 키워드
    "분위기 좋은 곳"    # 의미적
]

for query in test_queries:
    # Dense only
    dense_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=sentenceTransformer.encode(query).tolist(),
        using="dense",
        limit=3
    )

    print(f"\n🔹 Dense Only:")
        
    for hit in dense_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Sparse only
    query_tokens = query.split()
    sparse_scores = bm25.get_scores(query_tokens)
    sparse_idx = np.where(sparse_scores > 0)[0]

    sparse_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=sparse_idx.tolist(),
            values=sparse_scores[sparse_idx].tolist()
        ),
        using="sparse",
        limit=3
    )

    print(f"\n🔹 Sparse Only:")
    for hit in sparse_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Hybrid (DemRRF)
    hybrid_results = retrieve_similar_examples_hybrid(query, collection_name, top_k=3)

    print(f"\n🔹 Hybrid (Dense + Sparse + RRF):")
    for result in hybrid_results:
        print(f"  - {result['text'][:50]}... (Score: {result['score']:.4f})")

print("\n✅ 테스트 완료!")



📊 Dense vs Sparse vs Hybrid 비교

🔹 Dense Only:
  - 점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다.... (Score: 0.3593)
  - 인테리어는 북유럽 스타일로 밝고 감각적이며, 사진 촬영하기 좋은 곳으로 유명합니다.
   ... (Score: 0.2889)
  - 와인 페어링도 인기이며, 전문 소믈리에가 고기에 맞는 와인을 추천해줍니다.... (Score: 0.2777)

🔹 Sparse Only:
  - 미쉐린 3스타에 빛나는 '라떼르'는 서울 강남구 청담동에 위치한 프렌치 파인다이닝 레스토랑... (Score: 58.4669)
  - 한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처... (Score: 1.6381)

🔹 Hybrid (Dense + Sparse + RRF):
  - 한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처... (Score: 0.0318)
  - 미쉐린 3스타에 빛나는 '라떼르'는 서울 강남구 청담동에 위치한 프렌치 파인다이닝 레스토랑... (Score: 0.0313)
  - 점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다.... (Score: 0.0164)

🔹 Dense Only:
  - 참치, 광어, 성게 등 10가지 이상의 스시 코스를 제공합니다.... (Score: 0.6420)
  - 점심 세트 메뉴는 2만원대로 가성비가 뛰어납니다.... (Score: 0.6230)
  - 와인 페어링도 인기이며, 전문 소믈리에가 고기에 맞는 와인을 추천해줍니다.... (Score: 0.5889)

🔹 Sparse Only:
  - 미쉐린 3스타에 빛나는 '라떼르'는 서울 강남구 청담동에 위치한 프렌치 파인다이닝 레스토랑... (Score: 58.4669)
  - 한국의 식재료와 프렌치 테크닉을 결합한 독창적인 요리를 선보이고 있습니다.
    시그니처... (Score: 1.6381)

🔹 Hybrid